In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

from art.defences.trainer import AdversarialTrainerAWPPyTorch

import time

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            # 'TPR': '',
            # 'FNR': '',
            # 'TNR': '',
            # 'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/Defense/AWP/AWP.csv", mode='a', index=False)

In [3]:
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, f1_score,recall_score, roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def calculate_performance_metrics(x_test_adv, y_test, classifier, model_name, attack_name, epsilon):

    start_time = time.time()

    preds = np.argmax(classifier.predict(x_test_adv), axis=1)
    # preds = classifier.predict(x_test_adv)

    y_hat = preds
    # y_test_arg = np.argmax(y_test, axis=-1)
    y_test_arg = y_test


    accuracy = accuracy_score(y_test_arg, y_hat)
    precision_macro = precision_score(y_test_arg, y_hat, average='macro')
    # precision_micro = precision_score(y_test_arg, y_hat, average='micro')
    precision_weighted = precision_score(y_test_arg, y_hat, average='weighted')
    f1_macro = f1_score(y_test_arg, y_hat, average='macro')
    # f1_micro = f1_score(y_test_arg, y_hat, average='micro')
    f1_weighted = f1_score(y_test_arg, y_hat, average='weighted')
    recall_macro = recall_score(y_test_arg, y_hat, average='macro')
    # recall_micro = recall_score(y_test_arg, y_hat, average='micro')
    recall_weighted = recall_score(y_test_arg, y_hat, average='weighted')


    print(f"epsilon = {epsilon}")

    print(f"Accuracy: {accuracy}")
    # print(f"Precision(micro,macro,weighted): {precision_micro},{precision_macro},{precision_weighted}")
    # print(f"F1(micro,macro,weighted: {f1_micro},{f1_macro},{f1_weighted}")
    # print(f"Recall(micro,macro,weighted: {recall_micro},{recall_macro},{recall_weighted}")
    print(f"Precision(macro,weighted): {precision_macro},{precision_weighted}")
    print(f"F1(macro,weighted: {f1_macro},{f1_weighted}")
    print(f"Recall(macro,weighted: {recall_macro},{recall_weighted}")


    
    metrics = {
        "model" : model_name,
        "attack_model" : attack_name,
        "epsilon" : epsilon,
        "Accuracy": accuracy,
        # "Precision-micro": precision_micro,
        "Precision-macro": precision_macro,
        "Precision-weighted": precision_weighted,
        # "Recall_micro": recall_micro,
        "Recall_macro": recall_macro,
        "Recall_weighted": recall_weighted,
        
        # "F1-micro": f1_micro,
        "F1-macro": f1_macro,
        "F1-weighted": f1_weighted,

    }
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv("/home/jovyan/Defense/AWP/AWP.csv", mode='a', index=False, header=False)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time: {elapsed_time:.2f} second\n\n")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
x_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
x_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
x_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')


# train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

# val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
# val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [6]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [7]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [9]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/Wustl_iiot/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [10]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_val = x_val.astype(np.float32)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [12]:
from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent

FGSM = FastGradientMethod(estimator=classifier, eps=0.3)
BIM = BasicIterativeMethod(estimator=classifier, eps=0.3, max_iter=3)
PGD = ProjectedGradientDescent(estimator=classifier, eps=0.3, max_iter=3)

In [13]:
FGSM_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, FGSM, "TRADES", 0.1, 6.0, 0)
BIM_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, BIM, "TRADES", 0.1, 6.0, 0)
PGD_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, PGD, "TRADES", 0.1, 6.0, 0)

In [ ]:
# trining here, skip

In [13]:
start_time = time.time()
FGSM_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
                     validation_data=(x_val, y_val))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time: {elapsed_time:.2f} second\n\n")

Adversarial Training AWP with TRADES - Epochs: 100%|██████████| 20/20 [59:04<00:00, 177.21s/it]

Time: 3544.17 second




In [14]:
# start_time = time.time()
# BIM_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
#                      validation_data=(x_val, y_val))
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Time: {elapsed_time:.2f} second\n\n")

In [15]:
# start_time = time.time()
# PGD_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
#                      validation_data=(x_val, y_val))
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Time: {elapsed_time:.2f} second\n\n")

In [16]:
# calculate_performance_metrics(x_test, y_test, BIM_Trainer, 'BIM_Trainer', 'baseline', '0')
calculate_performance_metrics(x_test, y_test, FGSM_Trainer, 'FGSM_Trainer', 'baseline', '0')
# calculate_performance_metrics(x_test, y_test, PGD_Trainer, 'PGD_Trainer', 'baseline', '0')

epsilon = 0
Accuracy: 0.9982953093783115
Precision(macro,weighted): 0.5947964623438752,0.9979435892168569
F1(macro,weighted: 0.5763951699364166,0.9980405778895473
Recall(macro,weighted: 0.5620145631067961,0.9982953093783115
Time: 2.71 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'BIM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'FGSM', epsilon)

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, PGD_Trainer, 'PGD_Trainer', 'PGD', epsilon)

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.01
Accuracy: 0.9975288268629086
Precision(macro,weighted): 0.5923191328252375,0.9972863517591573
F1(macro,weighted: 0.5607801249689307,0.9972070501564786
Recall(macro,weighted: 0.5398058252427185,0.9975288268629086
Time: 1.54 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.1
Accuracy: 0.024611209073812684
Precision(macro,weighted): 0.13380173428184372,0.17813086546885487
F1(macro,weighted: 0.06718830202308065,0.03607870562058654
Recall(macro,weighted: 0.0914337628988501,0.024611209073812684
Time: 1.46 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.2
Accuracy: 0.0029737846226016006
Precision(macro,weighted): 0.014296737429416886,0.042264516975433646
F1(macro,weighted: 0.0031604479322787384,0.005482040411108736
Recall(macro,weighted: 0.0018638220656841786,0.0029737846226016006
Time: 1.46 second


epsilon = 0.3
Accuracy: 0.0002638710298928181
Precision(macro,weighted): 0.003527735226657179,0.0028232557301714222
F1(macro,weighted: 0.0008657407475840638,0.00047474217071601506
Recall(macro,weighted: 0.0004960035407988551,0.0002638710298928181
Time: 1.46 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
torch.save(FGSM_Trainer, "/home/jovyan/Defense/AWP/AWP_FGSM.pt")

In [ ]:
# skip star here

In [15]:
FGSM_Trainer = torch.load("/home/jovyan/Defense/AWP/AWP_FGSM.pt")



In [17]:
calculate_performance_metrics(x_test, y_test, FGSM_Trainer, 'FGSM_Trainer', 'baseline', '0')

epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
print("start BIM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'BIM', epsilon)

print("start FGSM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'FGSM', epsilon)

print("start PGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'PGD', epsilon)

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0
Accuracy: 0.9982953093783115
Precision(macro,weighted): 0.5947964623438752,0.9979435892168569
F1(macro,weighted: 0.5763951699364166,0.9980405778895473
Recall(macro,weighted: 0.5620145631067961,0.9982953093783115
Time: 2.61 second


start BIM


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.01
Accuracy: 0.997553957437184
Precision(macro,weighted): 0.591916560926691,0.997303888717741
F1(macro,weighted: 0.5611629805979546,0.9972365382985824
Recall(macro,weighted: 0.5405339805825242,0.997553957437184
Time: 2.59 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.1
Accuracy: 0.017344284679145394
Precision(macro,weighted): 0.03827254242877593,0.14584193032306275
F1(macro,weighted: 0.0148761523715992,0.02287461719614792
Recall(macro,weighted: 0.046139453721812315,0.017344284679145394
Time: 2.64 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.2
Accuracy: 0.0027727400283975487
Precision(macro,weighted): 0.008191357107280214,0.03648514962936827
F1(macro,weighted: 0.0012041785196946222,0.004784416343925809
Recall(macro,weighted: 0.0067888553714719165,0.0027727400283975487
Time: 2.61 second


epsilon = 0.3
Accuracy: 0.002550753288963908
Precision(macro,weighted): 0.008114842823629046,0.03635163808402137
F1(macro,weighted: 0.0010720214162057522,0.004766967037075477
Recall(macro,weighted: 0.0005739938299393346,0.002550753288963908
Time: 2.61 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

print("start DF")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'DF', epsilon)
    
# Iterate over epsilon values
print("start AutoPGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'AutoPGD', epsilon)

print("start BA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'BA', epsilon)

print("start HopSkipJump")
filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_HopSkipJump_eps_0.npy'
x_test_adv = np.load(filename)

calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'HopSkipJump', '0')

print("start ZOO")
ZOO_epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]
# Iterate over epsilon values
for epsilon in ZOO_epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'ZOO', epsilon)

start DF


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.01
Accuracy: 1.2565287137753244e-05
Precision(macro,weighted): 3.0230526355835674e-05,9.096036596913069e-05
F1(macro,weighted: 1.750337927859863e-05,1.3168152955172047e-05
Recall(macro,weighted: 2.644469960276353e-05,1.2565287137753244e-05
Time: 2.77 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.1
Accuracy: 1.2565287137753244e-05
Precision(macro,weighted): 3.0205434313858823e-05,9.08075345950202e-05
F1(macro,weighted: 1.751133405906842e-05,1.3170153635903032e-05
Recall(macro,weighted: 2.644469960276353e-05,1.2565287137753244e-05
Time: 2.62 second




...

epsilon = 0.1
Accuracy: 0.9981822217940717
Precision(macro,weighted): 0.593777550122324,0.9978295900788636
F1(macro,weighted: 0.5739471068859714,0.9979156854547531
Recall(macro,weighted: 0.5587378640776699,0.9981822217940717
Time: 2.62 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.2
Accuracy: 0.9981822217940717
Precision(macro,weighted): 0.593777550122324,0.9978295900788636
F1(macro,weighted: 0.5739471068859714,0.9979156854547531
Recall(macro,weighted: 0.5587378640776699,0.9981822217940717
Time: 2.61 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.3
Accuracy: 0.9981822217940717
Precision(macro,weighted): 0.593777550122324,0.9978295900788636
F1(macro,weighted: 0.5739471068859714,0.9979156854547531
Recall(macro,weighted: 0.5587378640776699,0.9981822217940717
Time: 2.61 second


